# Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
%matplotlib inline
from pandas.tseries.offsets import MonthEnd
from matplotlib.pylab import rcParams
pd.plotting.register_matplotlib_converters()
rcParams['figure.figsize'] = (15, 6)
import pmdarima as pm
print('done')

# Variables to Set

In [ ]:
data = 'forecasting_dataset.csv'
target_column = 'GDP ($ Billions)'
pred_date = '2021-04-13'

# Load/Transform Data

In [ ]:
df = pd.read_csv(data)
df.columns = df.columns.str.replace('DATEVALUE', 'Date')
df['Date'] = pd.to_datetime(df['Date'])
df.index = df['Date']
df = df.drop('Date',axis=1)
df

# Profile Data

In [ ]:
ProfileReport(df)

# Split into Training and Validation Data

In [ ]:
train = df[:int(0.75*(len(df)))]
valid = df[int(0.75*(len(df))):]

#plotting the data
ax = train[target_column].plot()
valid[target_column].plot(ax=ax)

# Stationarity
- Stationarity means that the statistical properties of the process do not change over time
- Random variables of the stochastic process remains the same as we shift it along the time index axis. (Mean and Variance)
- Stationary processes are easier to analyze

In [ ]:
from statsmodels.tsa.stattools import adfuller as adf
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()

    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
test_stationarity(df[target_column])

#### Analysis of Stationarity
- Even though there is an increasing trend for GDP, the rolling mean and standard deviations do not have large variances from year to year.

# Trends and Seasonality
 - Plot of Original Data
 - Plot of Smoothed Moving Average of Original Data
 - Plot of Seasonality
     - Seasonality is the presence of variations that occur at specific regular intervals less than a year, such as weekly, monthly, or quarterly
 - Plot of Residuals on a Simple Regression Fit
     - What is left over after fitting a model.

In [ ]:
#estimating trend and seasonlity
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(df[target_column])

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(df[target_column], label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

# ARIMA Modeling without Regressors (Only the Target)
- Autoregressive (AR) integrated (I) moving average (MA) models
- We are not going to go into a lot of detail on the parts of ARIMA, Differencing and Lags (This could be multiple college semester courses)
- Instead, what we will do is use Auto-Arima package to find the best model.

In [ ]:
best_noregressors = pm.auto_arima(train.iloc[:,0], 
                   trace=True, error_action='ignore', suppress_warnings=True,
                   seasonal=True,
                   m=12, #12 Months
                   stepwise=False,
                   D=None, 
                   max_D=10,
                   start_p=0, start_q=0, 
                   start_P=0, start_Q=0,
                   max_p=5, max_q=5, max_P=5, max_Q=5,
                   information_criterion='aic', #‘aic’, ‘bic’, ‘hqic’, ‘oob’
                   n_jobs = -1,
                  )

print(best_noregressors)
print('Order :',best_noregressors.order)
print('Seasonal Order :',best_noregressors.seasonal_order)
print('Intercept :',best_noregressors.with_intercept)
print(best_noregressors.summary())

# Fit Model and Make Predictions on Validation

In [ ]:
best_noregressors.fit(train.iloc[:,0])
best_noregressors_forecast = best_noregressors.predict(n_periods=len(valid), exogenous=True, return_conf_int=True,  alpha=0.1)

preds_best_noregressors = pd.merge(pd.DataFrame(best_noregressors_forecast[0],
                 index = pd.to_datetime(valid.index),
                 columns=['Prediction']),pd.DataFrame(best_noregressors_forecast[1][:,0],
                        index = pd.to_datetime(valid.index),
                        columns=['Lower']),how='left', left_index=True,right_index=True)

preds_best_noregressors = pd.merge(preds_best_noregressors,pd.DataFrame(best_noregressors_forecast[1][:,1],
                        index = pd.to_datetime(valid.index),
                        columns=['Upper']),how='left',left_index=True,right_index=True)

#plot the predictions for validation set
plt.plot(train.iloc[:,0], label='Train')
plt.plot(valid.iloc[:,0], label='Valid')
plt.plot(preds_best_noregressors.Prediction, label='Prediction')

plt.fill_between(preds_best_noregressors.index,preds_best_noregressors.Lower,preds_best_noregressors.Upper,color='k', alpha=.2)
plt.xlabel('Date')
plt.ylabel(target_column)
plt.legend(loc='best')
plt.show()

# ARIMA Modeling with Regressors (Target and Additional Features)
- exogenous parameter is the ony change

In [ ]:
best_regressors = pm.auto_arima(train.iloc[:,0], 
                   trace=True, error_action='ignore', suppress_warnings=True,
                   seasonal=True,
                   m=12, #12 Months
                   stepwise=False,
                   D=None, 
                   max_D=10,
                   start_p=0, start_q=0, 
                   start_P=0, start_Q=0,
                   max_p=5, max_q=5, max_P=5, max_Q=5,
                   information_criterion='aic', #‘aic’, ‘bic’, ‘hqic’, ‘oob’
                   n_jobs = -1,
                   exogenous=train.iloc[:,1:]
                  )

print(best_regressors)
print('Order :',best_regressors.order)
print('Seasonal Order :',best_regressors.seasonal_order)
print('Intercept :',best_regressors.with_intercept)
print(best_regressors.summary())

# Fit Model and Make Predictions on Validation

In [ ]:
best_regressors.fit(train.iloc[:,0])
best_regressors_forecast = best_regressors.predict(n_periods=len(valid), exogenous=True, return_conf_int=True,  alpha=0.1)

preds_best_regressors = pd.merge(pd.DataFrame(best_regressors_forecast[0],
                 index = pd.to_datetime(valid.index),
                 columns=['Prediction']),pd.DataFrame(best_regressors_forecast[1][:,0],
                        index = pd.to_datetime(valid.index),
                        columns=['Lower']),how='left', left_index=True,right_index=True)

preds_best_regressors = pd.merge(preds_best_regressors,pd.DataFrame(best_regressors_forecast[1][:,1],
                        index = pd.to_datetime(valid.index),
                        columns=['Upper']),how='left',left_index=True,right_index=True)

#plot the predictions for validation set
plt.plot(train.iloc[:,0], label='Train')
plt.plot(valid.iloc[:,0], label='Valid')
plt.plot(preds_best_regressors.Prediction, label='Prediction')

plt.fill_between(preds_best_regressors.index,preds_best_regressors.Lower,preds_best_regressors.Upper,color='k', alpha=.2)
plt.xlabel('Date')
plt.ylabel(target_column)
plt.legend(loc='best')
plt.show()

# Choose a ARIMA model and predict future

In [ ]:
pred_months = 12
model = best_regressors.fit(df.iloc[:,0])



forecast = model.predict(n_periods=pred_months, exogenous=True, return_conf_int=True,  alpha=0.25)
forecast

In [ ]:
preds = pd.merge(pd.DataFrame(forecast[0],columns=['Prediction']).reset_index(),
                 pd.DataFrame(forecast[1][:,0],columns=['Lower']),
                              how='left', left_index=True,right_index=True)
preds = pd.merge(preds,pd.DataFrame(forecast[1][:,1],columns=['Upper']),
                              how='left', left_index=True,right_index=True)

preds = preds.rename(columns={'index':'Date'})

preds = preds.set_index('Date')

display(preds)

In [ ]:
#plot the predictions for validation set
plt.plot(df.iloc[:,0], label='Previous')
plt.plot(preds.Prediction, label='Prediction')

plt.fill_between(preds.index,preds.Lower,preds.Upper,color='k', alpha=.2)
plt.xlabel('Date')
plt.ylabel('Terminations')
plt.legend(loc='best')
plt.show()